### Predicting formation enthalpies for solid solutions of Lanthanides Orthophosphates


In [1]:
import os
import sys
import inspect

print(sys.version)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)

sys.path.insert(0, grandparentdir) 

3.9.6 (default, Nov 10 2023, 13:38:27) 
[Clang 15.0.0 (clang-1500.1.0.2.5)]


In [2]:
import pandas as pd
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.optimize
import seaborn as sns
import scipy.stats as ss
from sklearn.linear_model import Lasso
from itertools import combinations, product
import itertools
import math
import read_data
import Utils
import lasso

## Monazite

#### Load data from files

In [3]:
list1 = read_data.readData("../../data/DATA_HE_monazite.dat")
print("Shape of List1 is {}".format(list1.shape))
print(list1)

Shape of List1 is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 1.29717527e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 1.90974365e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 1.92712564e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 2.12444086e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 2.22639048e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 2.09098752e-02]]


In [4]:
list2 = read_data.readCSVData("../../data/Data_Ln-monazite.csv", Volume=True)
print("Shape of List2 is {}".format(list2.shape))
print((list2))
list2 = list2[:,0:8]
print("Z, M, IP2, E. Charge, electronegativity, IP3, Vol")
print((list2))

Shape of List2 is (15, 9)
[[ 57.         138.90547     19.1773     139.1         11.
    1.1         49.95        77.61001595   1.216     ]
 [ 58.         140.116       20.198      146.5         12.
    1.12        36.758       75.78168414   1.196     ]
 [ 59.         140.90765     21.624      150.1         12.3
    1.13        38.98        74.20076494   1.179     ]
 [ 60.         144.242       22.1        154.1         12.95
    1.14        40.41        72.98174154   1.163     ]
 [ 61.         145.          22.3        157.4         13.6
    1.13        41.1         71.90916336   1.144     ]
 [ 62.         150.36        23.4        160.3         14.25
    1.17        41.4         70.88675537   1.132     ]
 [ 63.         151.964       24.92       163.1         14.9
    1.2         42.7         70.00137871   1.12      ]
 [ 64.         157.25        20.63       165.2         15.9
    1.2         44.          69.15236746   1.107     ]
 [ 65.         158.92535     21.91       164.5        

#### Generated Elemental Training/Testing sets from loaded data

In [5]:
def generateFeatures_full2(HEList, featureList):
    X = []
    Y = []
    
    for i in range(len(HEList)):
        Y.append(HEList[i][3])
        
        x = HEList[i][2]
        #Endmember 1
        X1 = featureList[int(HEList[i][0]-57)]
        #Endmember 2
        X2 = featureList[int(HEList[i][1]-57)]

        #X.append(np.concatenate((np.array([x, 1/x, x * x, 1 / (x * x), 1-x, 1/(1-x), (1-x) * (1-x), 1/( (1-x) * (1-x))]), np.array([abs(X1-X2)/2.0, 2.0 / (abs(X1-X2)), (X1 + X2) / 2.0, 2.0 / (X1 + X2)]).flatten())))
        # define the features related to m
        mfeatures = np.array([x, 1/x, x * x, 1 / (x * x), 1-x, 1/(1-x), (1-x) * (1-x), 1/( (1-x) * (1-x))])
        
        f1 = []
        f2 = []
        f3 = []
        f4 = []

        #for Z, M, IP2, E, Charge, generating features including AD, 1/AD, AM, 1 / AM
        for j in range(len(X1)-3):
            f1.append([abs(X1[j]-X2[j])/2.0])
            f2.append([2.0/abs(X1[j]-X2[j])])
            f3.append([(X1[j] + X2[j]) / 2.0])
            f4.append([2.0 / (X1[j] + X2[j])])
            
        #rint(f3)
        #print(np.asarray(f3).flatten())
    
        #print(np.concatenate((mfeatures, np.asarray(f1).flatten(), np.asarray(f3).flatten())))
              
        f5 = []
        f6 = []
        f7 = []

        #for electronegativity, only generating AD, AM, 1 / AM
        for j in range(5,6):
            f5.append([abs(X1[j]-X2[j])/2.0])
            f6.append([(X1[j] + X2[j]) / 2.0])
            f7.append([2.0 / (X1[j] + X2[j])])            


        f8 = []
        f10 = []
        f11 = []
        
        #for IP3, generating features including AD, 1/AD, AM, 1 / AM    
        for j in range(6,7):
            f8.append([abs(X1[j]-X2[j])/2.0])
            f10.append([(X1[j] + X2[j]) / 2.0])
            f11.append([2.0 / (X1[j] + X2[j])])
            
        f12 = []
        f13 = []
        f14 = []
        f15 = []
        f16 = []
        f17 = []
        f18 = []
        f19 = []
        f20 = []
        f21 = []
        f22 = []
        f23 = []
        
        #for Vol
        for j in range(7,len(X1)):
            f12.append([abs(X1[j]-X2[j])/2.0])
            f13.append([2.0/abs(X1[j]-X2[j])])
            f14.append([(X1[j] + X2[j]) / 2.0])
            f15.append([2.0 / (X1[j] + X2[j])])
            f16.append([pow(abs(X1[j]-X2[j])/2.0,2)])
            f17.append([pow(2.0/abs(X1[j]-X2[j]),2)])
            f18.append([pow((X1[j] + X2[j]) / 2.0,2)])
            f19.append([pow(2.0 / (X1[j] + X2[j]),2)])
            f20.append([pow(abs(X1[j]-X2[j])/2.0,3)])
            f21.append([pow(2.0/abs(X1[j]-X2[j]),3)])
            f22.append([pow((X1[j] + X2[j]) / 2.0,3)])
            f23.append([pow(2.0 / (X1[j] + X2[j]),3)])
            
        
        X.append(np.concatenate((mfeatures, np.asarray(f1).flatten(),  np.asarray(f2).flatten(),  np.asarray(f3).flatten(), np.asarray(f4).flatten(), np.asarray(f5).flatten(), np.asarray(f6).flatten(),np.asarray(f7).flatten(),np.asarray(f8).flatten(),np.asarray(f10).flatten(),np.asarray(f11).flatten(),np.asarray(f12).flatten(),np.asarray(f13).flatten(),np.asarray(f14).flatten(),np.asarray(f15).flatten(),np.asarray(f16).flatten(),np.asarray(f17).flatten(),np.asarray(f18).flatten(),np.asarray(f19).flatten(),np.asarray(f20).flatten(),np.asarray(f21).flatten(),np.asarray(f22).flatten(),np.asarray(f23).flatten()     )))
            
    return X, Y

In [6]:
listX, listY = generateFeatures_full2(list1, list2)
X = np.asarray(listX)
Y = np.asarray(listY)

print(X.shape)
print(Y.shape)

#elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Z]", "[AD:M]", "[AD:IP2]", "[AD:Young]", "[AD:Charge]",  "(1/[AD:Z])", "(1/[AD:M])", "(1/[AD:IP2])", "(1/[AD:Young])", "(1/[AD:Charge])",  "[AM:Z]", "[AM:M]", "[AM:IP2]", "[AM:Young]", "[AM:Charge]",  "(1/[AM:Z])", "(1/[AM:M])", "(1/[AM:IP2])", "(1/[AM:Young])", "(1/[AM:Charge])", "[AD:electronegativity]", "[AD:IP3]", "[AM:electronegativity]", "[AM:IP3]","(1/[AM:electronegativity])", "(1/[AM:IP3])" ,    "[AD:Vol]", "(1/[AD:Vol])","[AM:Vol]","(1/[AM:Vol])",   "([AD:Vol])^2", "(1/[AD:Vol])^2" , "([AM:Vol])^2","(1/[AM:Vol])^2",    "([AD:Vol])^3", "(1/[AD:Vol])^3", "([AM:Vol])^3", "(1/[AM:Vol])^3"         ]
elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Z]", "[AD:M]", "[AD:IP2]", "[AD:Young]", "[AD:Charge]",  "(1/[AD:Z])", "(1/[AD:M])", "(1/[AD:IP2])", "(1/[AD:Young])", "(1/[AD:Charge])",  "[AM:Z]", "[AM:M]", "[AM:IP2]", "[AM:Young]", "[AM:Charge]",  "(1/[AM:Z])", "(1/[AM:M])", "(1/[AM:IP2])", "(1/[AM:Young])", "(1/[AM:Charge])", "[AD:electronegativity]", "[AM:electronegativity]","(1/[AM:electronegativity])", "[AD:IP3]", "[AM:IP3]", "(1/[AM:IP3])" ,  "[AD:Vol]", "(1/[AD:Vol])","[AM:Vol]","(1/[AM:Vol])",   "([AD:Vol])^2", "(1/[AD:Vol])^2" , "([AM:Vol])^2","(1/[AM:Vol])^2",    "([AD:Vol])^3", "(1/[AD:Vol])^3", "([AM:Vol])^3", "(1/[AM:Vol])^3"         ]

print(len(elemental_list))
print(elemental_list)
m, n = X.shape

elemental_features = []

for i in elemental_list:
    elemental_features.append("("+i+")")
        
print("The elemental features are: \n {}".format(elemental_features))

(525, 46)
(525,)
46
['m', '1/m', 'm^2', '(1/m)^2', '(1-m)', '1/(1-m)', '(1-m)^2', '(1/(1-m))^2', '[AD:Z]', '[AD:M]', '[AD:IP2]', '[AD:Young]', '[AD:Charge]', '(1/[AD:Z])', '(1/[AD:M])', '(1/[AD:IP2])', '(1/[AD:Young])', '(1/[AD:Charge])', '[AM:Z]', '[AM:M]', '[AM:IP2]', '[AM:Young]', '[AM:Charge]', '(1/[AM:Z])', '(1/[AM:M])', '(1/[AM:IP2])', '(1/[AM:Young])', '(1/[AM:Charge])', '[AD:electronegativity]', '[AM:electronegativity]', '(1/[AM:electronegativity])', '[AD:IP3]', '[AM:IP3]', '(1/[AM:IP3])', '[AD:Vol]', '(1/[AD:Vol])', '[AM:Vol]', '(1/[AM:Vol])', '([AD:Vol])^2', '(1/[AD:Vol])^2', '([AM:Vol])^2', '(1/[AM:Vol])^2', '([AD:Vol])^3', '(1/[AD:Vol])^3', '([AM:Vol])^3', '(1/[AM:Vol])^3']
The elemental features are: 
 ['(m)', '(1/m)', '(m^2)', '((1/m)^2)', '((1-m))', '(1/(1-m))', '((1-m)^2)', '((1/(1-m))^2)', '([AD:Z])', '([AD:M])', '([AD:IP2])', '([AD:Young])', '([AD:Charge])', '((1/[AD:Z]))', '((1/[AD:M]))', '((1/[AD:IP2]))', '((1/[AD:Young]))', '((1/[AD:Charge]))', '([AM:Z])', '([AM:M]

In [7]:
dfX = pd.DataFrame(data=X, columns=elemental_features)

new_features=[]
new_columns = []

for i in range(n):
    for j in range(i):
        new_features.append(elemental_features[i]+"*"+elemental_features[j])
        new_columns.append(dfX[elemental_features[i]].values * dfX[elemental_features[j]].values)


for i in range(n):
    for j in range(i):
        for k in range(j):
            new_features.append(elemental_features[i]+"*"+elemental_features[j]+"*"+elemental_features[k])
            new_columns.append(dfX[elemental_features[i]] * dfX[elemental_features[j]] * dfX[elemental_features[k]])
            
            
new_columns = np.asarray(new_columns)
dfX = pd.concat(
    [
        dfX,
        pd.DataFrame(
            new_columns.T, 
            index=dfX.index, 
            columns=new_features
        )
    ], axis=1
)

dfX.head()

dfX.std() == 0
dfX=dfX.loc[:, dfX.std() > 0]
dfX.head()

,(m),(1/m),(m^2),((1/m)^2),((1-m)),(1/(1-m)),((1-m)^2),((1/(1-m))^2),([AD:Z]),([AD:M]),...,((1/[AM:Vol])^3)*(([AM:Vol])^3)*([AD:Vol]),((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AD:Vol])),((1/[AM:Vol])^3)*(([AM:Vol])^3)*([AM:Vol]),((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AM:Vol])),((1/[AM:Vol])^3)*(([AM:Vol])^3)*(([AD:Vol])^2),((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AD:Vol])^2),((1/[AM:Vol])^3)*(([AM:Vol])^3)*(([AM:Vol])^2),((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AM:Vol])^2),((1/[AM:Vol])^3)*(([AM:Vol])^3)*(([AD:Vol])^3),((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AD:Vol])^3)
0,0.750,1.333333,0.562500,1.777778,0.250,4.000000,0.062500,16.000000,0.5,0.605265,...,0.914166,1.093893,76.69585,0.013039,0.835699,1.196603,5882.253414,0.00017,0.763968,1.308956
1,0.625,1.600000,0.390625,2.560000,0.375,2.666667,0.140625,7.111111,0.5,0.605265,...,0.914166,1.093893,76.69585,0.013039,0.835699,1.196603,5882.253414,0.00017,0.763968,1.308956
2,0.500,2.000000,0.250000,4.000000,0.500,2.000000,0.250000,4.000000,0.5,0.605265,...,0.914166,1.093893,76.69585,0.013039,0.835699,1.196603,5882.253414,0.00017,0.763968,1.308956
3,0.375,2.666667,0.140625,7.111111,0.625,1.600000,0.390625,2.560000,0.5,0.605265,...,0.914166,1.093893,76.69585,0.013039,0.835699,1.196603,5882.253414,0.00017,0.763968,1.308956
4,0.250,4.000000,0.062500,16.000000,0.750,1.333333,0.562500,1.777778,0.5,0.605265,...,0.914166,1.093893,76.69585,0.013039,0.835699,1.196603,5882.253414,0.00017,0.763968,1.308956


In [8]:
print(len(dfX.columns.values))
np.array(dfX.columns.values)

16255


array(['(m)', '(1/m)', '(m^2)', ...,
       '((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AM:Vol])^2)',
       '((1/[AM:Vol])^3)*(([AM:Vol])^3)*(([AD:Vol])^3)',
       '((1/[AM:Vol])^3)*(([AM:Vol])^3)*((1/[AD:Vol])^3)'], dtype=object)

In [9]:
def LassoFit(lmb, X, Y, max_iter=100000, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
    lasso =  Lasso(alpha=lmb, max_iter=max_iter)
    lasso.fit(X_standardized, Y.copy())
    coef =  lasso.coef_
    selected_indices = coef.nonzero()[0]
    selected_features = np.array(dfX.columns.values)[selected_indices]
    Y_predict = lasso.predict(X_standardized)
    MAE, MSE, MAPE = Utils.compute_error(Y.copy(), Y_predict)
        
    return coef, selected_indices, selected_features, MAE, MSE, MAPE

LassoFit(0.01, dfX, Y)

(array([ 0., -0.,  0., ..., -0.,  0.,  0.]),
 array([ 1246,  1257,  5930,  6572,  7065,  7596,  9517,  9526,  9528,
        10066, 13099]),
 array(['([AD:Young])*((1-m))*(m)', '([AD:Young])*((1-m)^2)*(m^2)',
        '([AD:IP3])*((1/[AM:Charge]))*([AD:M])',
        '((1/[AM:IP3]))*([AD:M])*(1/(1-m))', '([AD:Vol])*((1-m))*(m)',
        '([AD:Vol])*((1/[AM:IP3]))*([AD:M])', '(([AD:Vol])^2)*((1-m))*(m)',
        '(([AD:Vol])^2)*((1-m)^2)*(m)', '(([AD:Vol])^2)*((1-m)^2)*(m^2)',
        '(([AD:Vol])^2)*((1/[AM:IP3]))*((1/[AM:Charge]))',
        '(([AD:Vol])^3)*((1/[AM:IP3]))*((1/[AD:Young]))'], dtype=object),
 0.050870640112123354,
 0.007019698499399649,
 0.12546941114263344)

In [10]:
#### Define a function which fits Lasso to have no more nonzero coefficients than a given threshold 
def LassoSelect(X, Y, min, max, step, threshold, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
        
    found = False
    for lmbda in np.arange (min, max, step):
        coef, selected_indices, selected_features, MAE, MSE, ME = LassoFit(lmbda, X.copy(), Y.copy())
        if len(selected_indices) <= threshold:
            found = True
            break
    
    if found:
        print("FOUND with threshold: {}".format(threshold))
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
            
    else:
        print("NOT FOUND with threshold: {}".format(threshold))
        print("Closest are: ")
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
     
    X_reduced = X[selected_features]
    
    return X_reduced


X_reduced = LassoSelect(dfX, Y, 0.001, 0.101, 0.005, 30)
X_reduced.head()

FOUND with threshold: 30
Lambda: 0.006, nnz: 17, MAE: 0.04838597343777562, MSE: 0.00611770613975703, MAPE: 0.12923974777896205


,([AD:M])*([AD:Z])*(1/(1-m)),([AD:Young])*((1/m)^2)*(1/m),([AD:Young])*((1-m))*(m),([AD:Charge])*([AD:M])*(1/(1-m)),([AD:electronegativity])*([AD:M])*(1/(1-m)),([AD:IP3])*((1/[AM:Charge]))*([AD:M]),([AD:Vol])*((1/[AM:IP3]))*([AD:M]),([AD:Vol])*((1/[AM:IP3]))*((1/[AM:Charge])),([AD:Vol])*((1/[AM:IP3]))*([AM:electronegativity]),(([AD:Vol])^2)*((1-m))*(m),(([AD:Vol])^2)*((1-m)^2)*(m),(([AD:Vol])^2)*((1-m)^2)*(m^2),(([AD:Vol])^2)*((1/[AM:IP3]))*((1/[AM:Charge])),(([AD:Vol])^3)*((1/m)^2)*(1/m),(([AD:Vol])^3)*((1/[AM:IP3]))*((1/[AD:Young])),(([AM:Vol])^3)*((1/[AM:Charge]))*([AD:M]),((1/[AM:Vol])^3)*((1-m)^2)*(m^2)
0,1.210530,8.770370,0.693750,1.210530,0.024211,0.347159,0.012763,0.001834,0.023406,0.156694,0.039173,0.029380,0.001676,1.810887,0.004763,23744.515727,7.792682e-08
1,0.807020,15.155200,0.867188,0.807020,0.016140,0.347159,0.012763,0.001834,0.023406,0.195867,0.073450,0.045906,0.001676,3.129212,0.004763,23744.515727,1.217607e-07
2,0.605265,29.600000,0.925000,0.605265,0.012105,0.347159,0.012763,0.001834,0.023406,0.208925,0.104462,0.052231,0.001676,6.111742,0.004763,23744.515727,1.385366e-07
3,0.484212,70.162963,0.867188,0.484212,0.009684,0.347159,0.012763,0.001834,0.023406,0.195867,0.122417,0.045906,0.001676,14.487093,0.004763,23744.515727,1.217607e-07
4,0.403510,236.800000,0.693750,0.403510,0.008070,0.347159,0.012763,0.001834,0.023406,0.156694,0.117520,0.029380,0.001676,48.893939,0.004763,23744.515727,7.792682e-08


In [11]:
def LassoL0(X, Y, nnz):    
    nr, nc = X.shape
    X = np.column_stack((X, np.ones(nr)))
    se_min = np.inner(Y, Y)
    coef_min, permu_min = None, None
    for permu in combinations(range(nc), nnz):
        X_ls = X[:, permu + (-1,)]
        coef, se, __1, __2 = np.linalg.lstsq(X_ls, Y, rcond=-1)
        try:
            if se[0] < se_min: 
                se_min = se[0]
                coef_min, permu_min = coef, permu
        except:
            pass
        
    return coef_min, permu_min

In [12]:
def LassoL0Fit(X_reduced, Y, nnz, log=True):
    
    scaler = StandardScaler()
    scaler.fit(X_reduced)
    X_std = scaler.transform(X_reduced)
    
    nr, nc = X_reduced.shape
   
    coefficients, selected_indices = LassoL0(X_std, Y, nnz)
   
    coefficients = np.array(coefficients)
    selected_indices = np.array(selected_indices)
    feature_reduced = np.array(X_reduced.columns.values)
    feature_list_selected = feature_reduced[selected_indices]
    
    X_selected = X_reduced[feature_list_selected]

    mean_selected = X_selected.mean()
    std_selected = X_selected.std()

    
    if log:
        print("Lasso: selected coefficients are: {}".format(coefficients))
        print("Lasso: selected features are: {}".format(feature_list_selected))
        
    #-mean/std
    mean_std = []
    for i in range(len(selected_indices)):
        mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
 
    sum_mean_std = sum(mean_std)

    for i in range(len(selected_indices)):
        coefficients[i] = coefficients[i] / std_selected[i]

    
    coefficients[len(selected_indices)] -= sum_mean_std
    
    function = str(coefficients[0])+" * "+feature_list_selected[0]
    
    for i in range(1, len(selected_indices)):
        if coefficients[i] >= 0:
            function += " + " + str(coefficients[i])+" * "+feature_list_selected[i]
        else:
            function += " - " + str(abs(coefficients[i]))+" * "+feature_list_selected[i]

    
    if coefficients[len(selected_indices)] >= 0:
        function += " + " + str(coefficients[len(selected_indices)])
    else:
        function += " - " + str(abs(coefficients[len(selected_indices)]))
    
    if log:
        print("Constructed function is: {}".format(function))

    X_selected = np.column_stack((X_selected, np.ones(X_selected.shape[0])))
    Y_predict = X_selected[:,0] * coefficients[0]

    for i in range(1,len(selected_indices)+1):
        Y_predict = Y_predict + X_selected[:,i] * coefficients[i]
    
    if log:
        Utils.print_error(Y.copy(),Y_predict,"Lasso L0: {} coef".format(nnz))
    
    return Y_predict, coefficients, selected_indices

In [13]:
LassoL0Fit(X_reduced, Y.copy(), 1);

Lasso: selected coefficients are: [1.9567842  1.65053546]
Lasso: selected features are: ['(([AD:Vol])^2)*((1-m))*(m)']
Constructed function is: 0.9246943428407507 * (([AD:Vol])^2)*((1-m))*(m) + 0.017339285855854758
Lasso L0: 1 coef
Mean absolute error: 0.062277848479921454
Mean squared error: 0.009550658795500384
Mean absolute percentage error: 0.19518882815326008


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [14]:
LassoL0Fit(X_reduced, Y.copy(), 2);

Lasso: selected coefficients are: [1.70222547 0.25707447 1.65053546]
Lasso: selected features are: ['(([AD:Vol])^2)*((1-m))*(m)' '(([AD:Vol])^2)*((1-m)^2)*(m^2)']
Constructed function is: 0.8044005355174457 * (([AD:Vol])^2)*((1-m))*(m) + 0.5302289190329357 * (([AD:Vol])^2)*((1-m)^2)*(m^2) + 0.022018422241566027
Lasso L0: 2 coef
Mean absolute error: 0.05756160536229981
Mean squared error: 0.008263524572486433
Mean absolute percentage error: 0.22413449495104704


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [15]:
LassoL0Fit(X_reduced, Y.copy(), 3);

Lasso: selected coefficients are: [ 1.83446989  0.18627821 -0.09820841  1.65053546]
Lasso: selected features are: ['(([AD:Vol])^2)*((1-m))*(m)' '(([AD:Vol])^2)*((1-m)^2)*(m)'
 '(([AD:Vol])^3)*((1/m)^2)*(1/m)']
Constructed function is: 0.8668937184314878 * (([AD:Vol])^2)*((1-m))*(m) + 0.16269073688728974 * (([AD:Vol])^2)*((1-m)^2)*(m) - 5.370699722264905e-05 * (([AD:Vol])^3)*((1/m)^2)*(1/m) + 0.010875461315375201
Lasso L0: 3 coef
Mean absolute error: 0.05163631631233237
Mean squared error: 0.006453105330510692
Mean absolute percentage error: 0.15662813852019433


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [16]:
LassoL0Fit(X_reduced, Y.copy(), 4);

Lasso: selected coefficients are: [ 0.06061413  1.77304079  0.19563707 -0.10368445  1.65053546]
Lasso: selected features are: ['([AD:Vol])*((1/[AM:IP3]))*((1/[AM:Charge]))'
 '(([AD:Vol])^2)*((1-m))*(m)' '(([AD:Vol])^2)*((1-m)^2)*(m)'
 '(([AD:Vol])^3)*((1/m)^2)*(1/m)']
Constructed function is: 24.653889522657398 * ([AD:Vol])*((1/[AM:IP3]))*((1/[AM:Charge])) + 0.837864896883438 * (([AD:Vol])^2)*((1-m))*(m) + 0.17086453576484342 * (([AD:Vol])^2)*((1-m)^2)*(m) - 5.670166406885661e-05 * (([AD:Vol])^3)*((1/m)^2)*(1/m) - 0.03372322627187252
Lasso L0: 4 coef
Mean absolute error: 0.05009123088663245
Mean squared error: 0.005875288423128829
Mean absolute percentage error: 0.1631372430256415


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [17]:
LassoL0Fit(X_reduced, Y.copy(), 5);

Lasso: selected coefficients are: [-0.06320061  0.09900865  1.69709481  0.24178568 -0.06324159  1.65053546]
Lasso: selected features are: ['([AD:Young])*((1/m)^2)*(1/m)'
 '([AD:Vol])*((1/[AM:IP3]))*((1/[AM:Charge]))'
 '(([AD:Vol])^2)*((1-m))*(m)' '(([AD:Vol])^2)*((1-m)^2)*(m)'
 '(([AD:Vol])^3)*((1/m)^2)*(1/m)']
Constructed function is: -0.00035270823203908095 * ([AD:Young])*((1/m)^2)*(1/m) + 40.27028039403801 * ([AD:Vol])*((1/[AM:IP3]))*((1/[AM:Charge])) + 0.8019760030241126 * (([AD:Vol])^2)*((1-m))*(m) + 0.21116957897800698 * (([AD:Vol])^2)*((1-m)^2)*(m) - 3.458477250159882e-05 * (([AD:Vol])^3)*((1/m)^2)*(1/m) - 0.03954786220312134
Lasso L0: 5 coef
Mean absolute error: 0.046048654264631866
Mean squared error: 0.0051649071979579814
Mean absolute percentage error: 0.17572796303444407


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18385/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]
